<a href="https://colab.research.google.com/github/Rish0x29/Python/blob/main/MNIST_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
print('Modules Imported')


Modules Imported


In [17]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64) # let's cast this number to an integer, as a float may cause an error along the way


# let's also store the number of test samples in a dedicated variable (instead of using the mnist_info one)
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)


def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.         # if we divide each element by 255, we would get the desired result -> all elements will be between 0 and 1


    return image, label


scaled_train_and_validation_data = mnist_train.map(scale) # .map() allows us to apply a custom transformation to a given dataset


test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000 # this BUFFER_SIZE parameter is here for cases when we're dealing with enormous datasets

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)


validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

train_data = shuffled_train_and_validation_data.skip(num_validation_samples)  #similarly, the train_data is everything else, so we skip as many samples as there are in the validation dataset


BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(num_test_samples)


validation_inputs, validation_targets = next(iter(validation_data))

In [18]:
input_size = 784
output_size = 10
# Use same hidden layer size for both hidden layers. Not a necessity.
hidden_layer_size = 50

# define how the model will look like
model = tf.keras.Sequential([

    # the first layer (the input layer)
    # each observation is 28x28x1 pixels, therefore it is a tensor of rank 3
    # since we don't know CNNs yet, we don't know how to feed such input into our net, so we must flatten the images
    # there is a convenient method 'Flatten' that simply takes our 28x28x1 tensor and orders it into a (None,)
    # or (28x28x1,) = (784,) vector
    # this allows us to actually create a feed forward neural network
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # input layer

    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation function
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer

    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])
# we define the optimizer we'd like to use,
# the loss function,
# and the metrics we are interested in obtaining at each iteration
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# determine the maximum number of epochs
NUM_EPOCHS = 5

# we fit the model, specifying the
# training data
# the total number of epochs
# and the validation data we just created ourselves in the format: (inputs,targets)
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose =2)

Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


540/540 - 12s - 21ms/step - accuracy: 0.8868 - loss: 0.4021 - val_accuracy: 0.9362 - val_loss: 0.2232
Epoch 2/5
540/540 - 6s - 11ms/step - accuracy: 0.9466 - loss: 0.1847 - val_accuracy: 0.9500 - val_loss: 0.1686
Epoch 3/5
540/540 - 9s - 16ms/step - accuracy: 0.9577 - loss: 0.1445 - val_accuracy: 0.9570 - val_loss: 0.1430
Epoch 4/5
540/540 - 5s - 10ms/step - accuracy: 0.9647 - loss: 0.1194 - val_accuracy: 0.9657 - val_loss: 0.1141
Epoch 5/5
540/540 - 4s - 7ms/step - accuracy: 0.9701 - loss: 0.1008 - val_accuracy: 0.9668 - val_loss: 0.1118
